In [1]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

In [2]:
!pip install - q tensorflow == 2.3

In [3]:
# аугментации изображений
!pip install albumentations - q

In [4]:
!pip install pymorphy2
!pip install pymorphy2-dicts

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import seaborn as sns
import random
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import pymorphy2
from nltk.corpus import stopwords
from datetime import datetime

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import *
from sklearn.model_selection import RandomizedSearchCV

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import *
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.applications.xception import Xception
import albumentations as a

# plt
import matplotlib.pyplot as plt
# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline


In [6]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)

In [7]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [8]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
VAL_SIZE=0.2
np.random.seed(RANDOM_SEED)

In [9]:
!pip freeze > requirements.txt

# 1. DATA

In [10]:
#from google.colab import drive
#drive.mount('/content/drive')

In [11]:
#import zipfile
#zip_file='/content/drive/MyDrive/cars.zip'
#z=zipfile.ZipFile(zip_file, 'r')
#z.extractall(path='/content/drive/MyDrive/cars/')
#print(os.listdir())

In [12]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')

In [13]:
train.info()

In [14]:
train.nunique()

In [15]:
train.isna().sum()

In [16]:
test.info()

In [17]:
test.nunique()

In [18]:
test.isna().sum()

In [19]:
# split данных
data_train, data_test = train_test_split(
    train, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

# 2. EDA и Feature-Engeneering

In [20]:
# посмотрим, как выглядят распределения числовых признаков
def visualize_distributions(titles_values_dict):
    columns = min(3, len(titles_values_dict))
    rows = (len(titles_values_dict) - 1) // columns + 1
    fig = plt.figure(figsize=(columns * 6, rows * 4))
    for i, (title, values) in enumerate(titles_values_dict.items()):
        hist, bins = np.histogram(values, bins=20)
        ax = fig.add_subplot(rows, columns, i + 1)
        ax.bar(bins[:-1], hist, width=(bins[1] - bins[0]) * 0.7)
        ax.set_title(title)
    plt.show()


visualize_distributions({
    'mileage': train['mileage'].dropna(),
    'modelDate': train['modelDate'].dropna(),
    'productionDate': train['productionDate'].dropna()
})

## 3.1. Предобработка табличных данных

In [21]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1  # трейн
test['sample'] = 0  # тест
test['price'] = 0  # в тесте нет значения price,  заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True)  # объединяем
print(train.shape, test.shape, data.shape)

In [22]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [23]:
profile = ProfileReport(data, title='data_report', html={'style':{'full_width':True}})

In [24]:
profile.to_notebook_iframe()

In [25]:
profile.to_file(output_file="data_report.html")

### 3.1.1. bodyType

In [26]:
data.bodyType.unique()

In [27]:
# Выбираем первое слово для описания типа кузова
data['bodyType'] = data['bodyType'].astype(
    str).apply(lambda x: None if x.strip() == '' else x)
# Понижаем регистр первого слова
data['bodyType'] = data.bodyType.apply(lambda x: x.split(' ')[0].lower())

In [28]:
train.bodyType.unique()

In [29]:
data.bodyType.value_counts().plot.barh()

### 3.1.2. brand

In [30]:
data.brand.unique()

In [31]:
data.brand.value_counts().plot.barh()

### 3.1.3. color

In [32]:
data.color.nunique()

In [33]:
data.color.value_counts().plot.barh()

### 3.1.4. description

In [34]:
data['comment_length'] = data.description.apply(lambda x: len(str(x)))

In [35]:
data.comment_length.hist()

### 3.1.5. engineDisplacement

In [36]:
data.engineDisplacement.unique()

In [37]:
data.engineDisplacement = data.engineDisplacement.apply(lambda x: x[:3])

In [38]:
data.engineDisplacement.value_counts()

In [39]:
data.engineDisplacement = data.engineDisplacement.replace(
    'und', data.engineDisplacement.mode()[0])

In [40]:
data.engineDisplacement.unique()

In [41]:
data.engineDisplacement = data.engineDisplacement.apply(lambda x: float(x))

In [42]:
data.engineDisplacement.hist()

### 3.1.6. enginePower

In [43]:
data.enginePower.unique()

In [44]:
# берем 1 символ, если длина строки 5, первые два если 6,в противном случае первые три символа
data['enginePower'] = data['enginePower'].apply(
    lambda x: x[:1] if len(x) == 5 else (x[:2] if len(x) == 6 else x[:3]))

In [45]:
data.enginePower.unique()

In [46]:
data['enginePower'] = data['enginePower'].apply(lambda x: int(x))
data.enginePower.hist()

### 3.1.7. fuelType

In [47]:
data.fuelType.value_counts()

In [48]:
data.fuelType.hist()

### 3.1.8. mileage

In [49]:
data.mileage.unique()

In [50]:
data.mileage.hist(figsize=(8, 5), bins=100)

In [51]:
IQR = data['mileage'].quantile(0.75) - data['mileage'].quantile(0.25)
perc25 = data['mileage'].quantile(0.25)  # 25-й перцентиль
perc75 = data['mileage'].quantile(0.75)  # 75-й перцентиль

print(
    '25-й перцентиль: {},'.format(perc25),
    '75-й перцентиль: {},'.format(perc75),
    "IQR: {}, ".format(IQR),
    "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR,
                                           l=perc75 + 1.5*IQR))

In [52]:
data.mileage[data.mileage > 328841].count()

### 3.1.9. modelDate

In [53]:
data.modelDate.unique()

In [54]:
data.modelDate.hist()

In [55]:
data['model_time'] = datetime.now().year - data.modelDate

### 3.1.10 model_info

In [56]:
data.model_info.value_counts()

In [57]:
data.loc[data['model_info'] == 'None']

In [58]:
data.iloc[2803]['description']

### 3.1.11. name

In [59]:
data['xDrive'] = data['name'].apply(lambda x: 1 if 'xDrive' in x else 0)

In [60]:
sns.countplot(x='xDrive', data=data)

### 3.1.12. numberOfDoors

In [61]:
data.numberOfDoors.unique()

In [62]:
sns.countplot(x='numberOfDoors', data=data)

### 3.1.13. productionDate

In [63]:
data[['modelDate', 'productionDate']].corr()

### 3.1.14. sell_id

### 3.1.15. vehicleConfiguration

### 3.1.16. vehicleTransmission

Признак содержит информацию о типе коробки передач, используемой в транспортном средстве. Проверим уникальные значения:

In [64]:
data.vehicleTransmission.unique()

In [65]:
sns.countplot(x='vehicleTransmission', data=data)

### 3.1.17. Владельцы

In [66]:
data.Владельцы.isna().sum()

In [67]:
data[data['Владельцы'].isnull()]

In [68]:
data.iloc[6665]['description']

In [69]:
data.Владельцы = data.Владельцы.apply(
    lambda x: data.Владельцы.mode()[0] if pd.isna(x) else x)

In [70]:
data.Владельцы.isna().sum()

In [71]:
data['Владельцы'] = data['Владельцы'].apply(
    lambda x: int(x[0])).astype('int32')

In [72]:
data.Владельцы.hist()

### 3.1.18. Владение

In [73]:
data.Владение.isna().sum()

In [74]:
data.Владение.isna().sum()/len(data.Владение)*100

### 3.1.19. ПТС

In [75]:
data.ПТС.unique()

In [76]:
data.ПТС.hist()

### 3.1.20. Привод

In [77]:
sns.countplot(x='Привод', data=data)

### 3.1.21. Руль

In [78]:
sns.countplot(x='Руль', data=data)

### 3.1.22. Целевой признак - price

In [79]:
data.iloc[6665]['price']

In [80]:
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.title(f"Распределение столбца {'price'}")
sns.distplot((data[data['sample'] == 1]['price']), bins=50)

plt.subplot(1, 2, 2)
sns.boxplot(data['price'])
plt.xlabel('Price')
plt.title(f"Боксплот столбца {'price'}", fontsize=12)
plt.show()
data.price.describe()

In [81]:
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.title(f"Распределение после log.{'price'} ")
sns.distplot(np.log(data[data['sample'] == 1]['price']), bins=50)

plt.subplot(1, 2, 2)
sns.boxplot(np.log(data[data['sample'] == 1]['price']))
plt.xlabel('Price')
plt.title('Боксплот после log.price', fontsize=12)
plt.show()

In [82]:
plt.figure(figsize=(10, 5))
plt.scatter((data.price), data.model_time)

In [83]:
plt.figure(figsize=(8, 5))
plt.scatter((data.price), data.Привод)

In [84]:
# используем все текстовые признаки как категориальные без предобработки
cat_features = ['bodyType', 'brand', 'color', 'fuelType', 'model_info',
                'numberOfDoors', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'xDrive']

# числовые признаки
num_features = ['mileage', 'modelDate', 'model_time',
                'engineDisplacement', 'enginePower', 'comment_length']

In [85]:
plt.figure(figsize=(12, 7))
sns.heatmap(data[data['sample'] == 1]
            [num_features + ['price']].corr(), annot=True)

In [86]:
data[['model_time', 'productionDate']].corr()

In [87]:
# числовые признаки
num_features = ['mileage', 'model_time', 'productionDate',
                'engineDisplacement', 'enginePower', 'comment_length']

In [88]:
def preproc_data(df_input):

    df_output = df_input.copy()

    # Удалим неиспользуемые столбцы
    df_output.drop(['description', 'sell_id', 'vehicleConfiguration',
                    'Владение', 'name', 'modelDate'], axis=1, inplace=True)

    ############################### Нормализация ####################################################################

    scaler = RobustScaler()  # показал наилучший результат из всех scaler'ов
    for column in num_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:, 0]

    #################### Работа с категориальными признаками ############################################################
    # Label Encoding
    for column in cat_features:
        df_output[column], _ = pd.factorize(df_output[column])

    # One-Hot Encoding:
    df_output = pd.get_dummies(
        df_output, columns=cat_features, dummy_na=False)

    return df_output

In [89]:
# Запускаем и проверяем, что получилось
df_preproc = preproc_data(data)
df_preproc.sample(10)

## 3.2. Split data

In [90]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

In [91]:
test_data.info()

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

## 4 ML Model

In [93]:
estimators = [('gbr', GradientBoostingRegressor(n_estimators=200, 
                            min_samples_split=10,
                            min_samples_leaf=1,
                            max_features='sqrt',
                            max_depth=100, 
                            learning_rate=0.1111111111111111, 
                            random_state=RANDOM_SEED)),
             ('rfr', RandomForestRegressor(random_state=RANDOM_SEED,
                            n_estimators=600,
                            min_samples_split=2,
                            min_samples_leaf=1,
                            max_samples=1.0,
                            max_features='auto',
                            max_depth=20,
                            bootstrap=True))]
sr = StackingRegressor(estimators=estimators,
                     final_estimator=None,
                     n_jobs=1, 
                     cv=2, 
                     passthrough= False)
sr.fit(X_train, np.log(y_train))
predict_test_ml = np.exp(sr.predict(X_test))

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test_ml))*100:0.2f}%")

In [94]:
sub_predict_ml = np.exp(sr.predict(X_sub))
sample_submission['price'] = sub_predict_ml
sample_submission.to_csv('ml_sub.csv', index=False)

In [95]:
#---------------------------------------------------------

In [96]:
# Укажем размер выводимого изображения
plt.figure(figsize=(10, 6))
# 9 случайных примеров из train
random_image = train.sample(n=9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values
# выведем 9 изображений автомобилей и цен к ним
for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [97]:
# Установим размер изображения
size = (320, 240)
# функция для загрузки изображений


def get_image_array(index):
    images_set = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)  # изменение размера
        images_set.append(image)  # добавляем изображение в массив
    images_set = np.array(images_set)
    print('images shape', images_set.shape, 'dtype', images_set.dtype)
    return(images_set)


# применим функцию для создания выборок
images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

In [98]:
augment_module = a.Compose([
    # добавляем размытие по Гауссу и шум с вероятностью 7%
    a.Blur(p=0.07),
    a.GaussNoise(p=0.07),
    #  Установим параметры сдвига,поворота и масштабирования, а также укажем их вероятность.
    a.ShiftScaleRotate(shift_limit=0.08,
                       scale_limit=0.05,
                       border_mode=4,
                       rotate_limit=20,
                       p=0.7),

    a.RGBShift(),
    a.HueSaturationValue(),  # случайный оттенок и насыщенность
    a.HorizontalFlip(),

    # установим случайную яркость и контрастность изображений с вероятностью 50%

    a.OneOf([
            a.RandomBrightnessContrast(
                brightness_limit=0.3, contrast_limit=0.3),
            a.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)],
            p=0.3)
])

# Выведем пример аугментации
plt.figure(figsize=(12, 8))
for i in range(9):
    img = augment_module(image=images_train[0])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [99]:
morphy = pymorphy2.MorphAnalyzer()
df_NLP = data.copy()

In [100]:
# Паттерн с символами
trash_sym = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-–»«•∙·✔➥●☛“”°№₽®]+"

# функция для лемматизации текста:


def lemma(text):
    text = text.lower()  # понижаем регистр
    text = re.sub(trash_sym, ' ', text)  # удаляем символы из паттерна
    strings = []  # создаем массив, в котором будут храниться лемматизированные строки
    for wrd in text.split():  # берем слово из строки
        wrd = wrd.strip()  # убираем пробелы до и после слова
        wrd = morphy.normal_forms(wrd)[0]  # приводим к нормальной форме
        strings.append(wrd)  # добавляем слово в строку массива
    return ' '.join(strings)  # вернем значения, разделив пробелами

In [101]:
strings_set = []
strings_set = df_NLP.apply(
    lambda df_NLP: lemma(df_NLP.description), axis=1)

In [102]:
russian_stopwords = stopwords.words("russian")

In [103]:
# функция для проверки на стоп-слова
def lineWithoutStopWords(line):
    line = line.split()  # разделяем на слова
    # возвращаем слово, если оно не в списке стоп-слов
    return [word for word in line if word not in russian_stopwords]


# применим функцию к нашим лемматизированым строкам слов
str_without_stop = [lineWithoutStopWords(line) for line in strings_set]

In [104]:
# split данных
text_train = data.description.iloc[X_train.index]
text_test = data.description.iloc[X_test.index]
text_sub = data.description.iloc[X_sub.index]

In [105]:
# The maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 256

In [106]:
# обучение токенизатора для NLP
tokenize = Tokenizer(num_words=MAX_WORDS)
tokenize.fit_on_texts(str_without_stop)

In [107]:
# аугментация изображений
def process_image(image):
    return augment_module(image=image.numpy())['image']
# векторизация строки


def tokenize_(descriptions):
    return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen=MAX_SEQUENCE_LENGTH)
# применение векторизации к тексту


def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]
# функция  для применения вышеупомянутых функций к входным данным обучающей выборки


def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image, ] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text, ] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price
# функция  для применения вышеупомянутых функций к входным данным валидационной и тестовой выборкок


def tf_process_val_dataset_element(image, table_data, text, price):
    [text, ] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price


# использование tf.data.Dataset с использованием функций для обучающей выборки
train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, data.description.iloc[X_train.index], y_train
)).map(tf_process_train_dataset_element)
# использование tf.data.Dataset с использованием функций для валидационной выборки
test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, data.description.iloc[X_test.index], y_test
)).map(tf_process_val_dataset_element)
# использование tf.data.Dataset с использованием функций для тестовой выборки
y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, data.description.iloc[X_sub.index], y_sub
)).map(tf_process_val_dataset_element)

# проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__()
test_dataset.__iter__().__next__()
sub_dataset.__iter__().__next__()

In [108]:
# загрузим модель без "головы" и укажем, что она может обучаться
xception_model = tf.keras.applications.xception.Xception(
    weights='imagenet', include_top=False, input_shape=(size[1], size[0], 3))
xception_model.trainable = True

In [109]:
# Заморозим 1\4 нижних слоев модели
for layer in xception_model.layers[:len(xception_model.layers)//4]:
    layer.trainable = False

In [110]:
print(len(xception_model.layers))

for layer in xception_model.layers:
    print(layer, layer.trainable)

In [111]:
xception_output = L.GlobalAveragePooling2D()(xception_model.output)

In [112]:
# Табличная нейронная сеть
tabular_model = Sequential([
    L.Input(shape=X.shape[1]),
    L.Dense(512, input_dim=X_train.shape[1], activation="relu"),
    L.Dropout(0.5),
    L.Dense(256, kernel_regularizer=regularizers.l2(
        l2=1e-6), activation="relu"),
    L.Dropout(0.5),
    L.Dense(128, kernel_regularizer=regularizers.l2(
        l2=1e-5), activation="relu"),
    L.Dropout(0.25)
])

In [113]:
# Нейронная сеть для NLP
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LayerNormalization(),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.5),
    L.Dense(128, activation="sigmoid"),
    L.Dropout(0.5),
    L.LSTM(64,),
    L.Dropout(0.25),
    L.Dense(64, activation="relu"),
    L.Dropout(0.25),
])

In [115]:
# объединяем выходы трех нейросетей
combinedInput = L.concatenate(
    [xception_output, tabular_model.output, nlp_model.output])

# being our regression head
head = L.Dense(128, activation="relu")(combinedInput)
head = L.Dense(1,)(head)
# Соберем наши части в одну модель
model = Model(inputs=[xception_model.input,
                      tabular_model.input, nlp_model.input], outputs=head)
# посмотрим описание нашей модели
model.summary()

In [116]:
# укажем используемый оптимизатор и начальную скорость обучения
optimizer = tf.keras.optimizers.Adam(0.01)
# компиляция модели
model.compile(loss='MAPE', optimizer=optimizer, metrics=['MAPE'])

In [117]:
checkpoint = ModelCheckpoint(
    '../working/best_model.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
earlystop1 = EarlyStopping(
    monitor='val_MAPE', patience=10, restore_best_weights=True,)
lr_scheduler1 = ReduceLROnPlateau(monitor='val_loss',
                                  factor=0.5,  # уменьшим lr в 2 раза
                                  patience=3,  # если нет улучшения через 3 эпохи - уменьшить lr
                                  min_lr=0.00001,  # минимальная скорость обучения
                                  verbose=1,  # выводить сообщения об уменьшении скорости
                                  mode='auto')  # выбранный способ отслеживания метрики
callbacks_list1 = [checkpoint, earlystop1, lr_scheduler1]

In [118]:
history = model.fit(train_dataset.batch(30),
                    epochs=100,
                    validation_data=test_dataset.batch(30),
                    callbacks=callbacks_list1
                    )

In [119]:
plt.title('Loss')
plt.plot(history.history['MAPE'], label='train')
plt.plot(history.history['val_MAPE'], label='test')
plt.show()

In [120]:
model.load_weights('../working/best_model.hdf5')
model.save('../working/nn_final.hdf5')

In [121]:
test_predict_nn = model.predict(test_dataset.batch(30))
print(f"TEST mape: {(mape(y_test, test_predict_nn[:,0]))*100:0.2f}%")

In [122]:
sub_predict_nn = model.predict(sub_dataset.batch(30))
sample_submission['price'] = sub_predict_nn[:, 0]
sample_submission.to_csv('nn_submission.csv', index=False)

In [123]:
blend_predict = (predict_test_ml + test_predict_nn[:, 0]) / 2
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [124]:
blend_sub_predict = (sub_predict_ml + sub_predict_nn[:, 0]) / 2
sample_submission['price'] = blend_sub_predict
sample_submission.to_csv('blend_submission.csv', index=False)